### Import

- 0.741 : cat_0.7409에서 원핫인코딩 추가 / 특정 시술유형 추가 / 랜덤 배정 전처리 삭제
- 0.7394 : 0.741에서 컬럼 삭제 시도
- 0.74105: 0.741에서 특정 시술 유형 전처리 삭제

In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from google.colab import drive
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Data Load

In [5]:
train = pd.read_csv('/content/drive/MyDrive/데이터 분석/LG Aimer/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/데이터 분석/LG Aimer/test.csv').drop(columns=['ID'])

In [6]:
categorical_columns = [
      "시술 시기 코드",
    "시술 당시 나이",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
      "난자 나이"
]

In [7]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일",
    "시술 유형_DI",
    "시술 유형_IVF"
]

In [8]:
# '배아 이식 경과일' 결측치를 평균값으로 채우기
train["배아 이식 경과일"] = train["배아 이식 경과일"].fillna(train["배아 이식 경과일"].mean())
test["배아 이식 경과일"] = test["배아 이식 경과일"].fillna(test["배아 이식 경과일"].mean())

In [9]:
train['특정 시술 유형'] = train['특정 시술 유형'].fillna('ICSI')
train['특정 시술 유형'].isna().sum()

0

In [10]:
test['특정 시술 유형'] = test['특정 시술 유형'].fillna('ICSI')
test['특정 시술 유형'].isna().sum()

0

In [11]:
train['난자 출처'] = train['난자 출처'].replace("알 수 없음", "본인 제공")
train['난자 출처'].value_counts()

,count
난자 출처,
본인 제공,240582
기증 제공,15769


In [12]:
test['난자 출처'] = test['난자 출처'].replace("알 수 없음", "본인 제공")
test['난자 출처'].value_counts()

,count
난자 출처,
본인 제공,84573
기증 제공,5494


In [13]:
# '난자 나이' 컬럼 생성 (초기값을 모두 '만18-34세'로 설정)
train['난자 나이'] = '만18-34세'
test['난자 나이'] = '만18-34세'

In [14]:
# 기증자 제공 → 난자 나이를 난자 기증자 나이로 설정
if '난자 기증자 나이' in train.columns:
    train.loc[train['난자 출처'] == '기증자 제공', '난자 나이'] = train['난자 기증자 나이'].astype(str)
if '난자 기증자 나이' in test.columns:
    test.loc[test['난자 출처'] == '기증자 제공', '난자 나이'] = test['난자 기증자 나이'].astype(str)

# 본인 제공 → 난자 나이를 시술 당시 나이로 설정
if '시술 당시 나이' in train.columns:
    train.loc[train['난자 출처'] == '본인 제공', '난자 나이'] = train['시술 당시 나이'].astype(str)
if '시술 당시 나이' in test.columns:
    test.loc[test['난자 출처'] == '본인 제공', '난자 나이'] = test['시술 당시 나이'].astype(str)

In [15]:
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['IUI', 'ICI', 'Generic DI', 'IVI', 'GIFT'], 'DI')
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['FER'], 'IVF')
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['ICSI:Unknown'], 'ICSI')
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['IVF:Unknown'], 'IVF')
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['ICSI / BLASTOCYST:IVF / BLASTOCYST','ICSI / BLASTOCYST :IVF / BLASTOCYST', 'ICSI / BLASTOCYST :ICSI'], 'ICSI / BLASTOCYST')
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['IVF / AH:ICSI / AH'], 'IVF / AH')
# train['특정 시술 유형'] = train['특정 시술 유형'].replace(['ICSI / AH:Unknown'], 'ICSI / AH')

# test['특정 시술 유형'] = test['특정 시술 유형'].replace(['IUI', 'ICI', 'Generic DI', 'IVI', 'GIFT','IUI:ICI'], 'DI')
# test['특정 시술 유형'] = test['특정 시술 유형'].replace(['ICSI / BLASTOCYST:IVF / BLASTOCYST'], 'ICSI')
# test['특정 시술 유형'] = test['특정 시술 유형'].replace(['IVF:Unknown:Unknown:Unknown'], 'IVF')
# test['특정 시술 유형'] = test['특정 시술 유형'].replace(['ICSI:Unknown'], 'ICSI')
# test['특정 시술 유형'] = test['특정 시술 유형'].replace(['IVF:Unknown'], 'IVF')

In [16]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder 생성
encoder = OneHotEncoder(sparse_output=False, dtype=int)

# 원핫 인코딩 적용
encoded_array0 = encoder.fit_transform(train[['시술 유형']])
encoded_array1 = encoder.fit_transform(test[['시술 유형']])

# 새로운 데이터프레임 생성
encoded_df0 = pd.DataFrame(encoded_array0, columns=encoder.get_feature_names_out(['시술 유형']))
encoded_df1= pd.DataFrame(encoded_array1, columns=encoder.get_feature_names_out(['시술 유형']))

# 원본 컬럼 삭제 후 새로운 컬럼 추가
train = pd.concat([train.drop(columns=['시술 유형']), encoded_df0], axis=1)
test = pd.concat([test.drop(columns=['시술 유형']), encoded_df1], axis=1)

In [17]:
train['총 임신 횟수'] = train['총 임신 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
train['IVF 임신 횟수'] = train['IVF 임신 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
train['DI 임신 횟수'] = train['DI 임신 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})

train['총 출산 횟수'] = train['총 출산 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
train['IVF 출산 횟수'] = train['IVF 출산 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
train['DI 출산 횟수'] = train['DI 출산 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})




test['총 임신 횟수'] = test['총 임신 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
test['IVF 임신 횟수'] = test['IVF 임신 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
test['DI 임신 횟수'] = test['DI 임신 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})

test['총 출산 횟수'] = test['총 출산 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
test['IVF 출산 횟수'] = test['IVF 출산 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})
test['DI 출산 횟수'] = test['DI 출산 횟수'].replace({'0회': '0회', '1회': '1회',
                                              '2회': '2회 이상', '3회': '2회 이상',
                                              '4회': '2회 이상', '5회': '2회 이상',
                                              '6회 이상': '2회 이상'})

In [18]:
train[categorical_columns] = train[categorical_columns].fillna(0)
train[numeric_columns] = train[numeric_columns].fillna(0)
test[categorical_columns] = test[categorical_columns].fillna(0)
test[numeric_columns] = test[numeric_columns].fillna(0)

In [19]:
'''
# 삭제할 컬럼 목록 (시술 당시 나이는 삭제 X)
cols_to_drop = [
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불임 원인 - 남성 요인",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "정자 출처", "정자 기증자 나이"


]
'''

'\n# 삭제할 컬럼 목록 (시술 당시 나이는 삭제 X)\ncols_to_drop = [\n    "남성 주 불임 원인",\n    "남성 부 불임 원인",\n    "부부 주 불임 원인",\n    "부부 부 불임 원인",\n    "불임 원인 - 남성 요인",\n    "불임 원인 - 정자 면역학적 요인",\n    "불임 원인 - 정자 운동성",\n    "불임 원인 - 정자 형태",\n    "정자 출처", "정자 기증자 나이"\n\n\n]\n'

In [20]:
'''
# 불필요한 컬럼 삭제
train.drop(columns=cols_to_drop, errors='ignore', inplace=True)
test.drop(columns=cols_to_drop, errors='ignore', inplace=True)
'''

"\n# 불필요한 컬럼 삭제\ntrain.drop(columns=cols_to_drop, errors='ignore', inplace=True)\ntest.drop(columns=cols_to_drop, errors='ignore', inplace=True)\n"

### Data Pre-processing

In [21]:
X = train.drop(columns=['임신 성공 여부'])  # 'target'은 예시, 실제 종속 변수명을 사용
y = train['임신 성공 여부']

In [22]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

In [23]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [24]:
#로그변환 후 표준화

# 전처리할 열 리스트
columns_to_transform = [
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수"
]

# 1. 로그 변환 (0 값 처리를 위해 log1p 사용)
X_train_encoded[columns_to_transform] = X_train_encoded[columns_to_transform].apply(lambda x: np.log1p(x))
X_test_encoded[columns_to_transform] = X_test_encoded[columns_to_transform].apply(lambda x: np.log1p(x))

# 2. 표준화 (Z-score scaling)
scaler = StandardScaler()
X_train_encoded[columns_to_transform] = scaler.fit_transform(X_train_encoded[columns_to_transform])
X_test_encoded[columns_to_transform] = scaler.transform(X_test_encoded[columns_to_transform])


In [25]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# 범주형 컬럼을 문자열(str)로 변환
X_train_encoded[categorical_columns] = X_train_encoded[categorical_columns].astype(str)
X_test_encoded[categorical_columns] = X_test_encoded[categorical_columns].astype(str)

In [26]:
# 숫자형 변수를 처리할 열 목록
columns_to_process = [
   "시술 시기 코드",
    "시술 당시 나이",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
   "난자 나이"
]

def process_count(value):
    # 값이 None이거나 공백 문자열이면 기본값 0을 반환
    if value is None or str(value).strip() == "":
        return 0
    try:
        # 직접 정수 변환 시도
        return int(value)
    except (ValueError, TypeError):
        # 문자열에서 숫자만 추출
        digits = ''.join(filter(str.isdigit, str(value)))
        # 추출된 결과가 비어있으면 0 반환, 아니면 정수로 변환
        return int(digits) if digits else 0

# 예시로 모든 숫자형 변수를 변환하는 코드
for col in columns_to_process:
    X_train_encoded[col] = X_train_encoded[col].apply(process_count)
    X_test_encoded[col] = X_test_encoded[col].apply(process_count)


In [27]:
# # 이거 넣으니까 떨어짐
# drop_columns = [
#     "정자 기증자 나이",
#     "정자 출처",
#     "대리모 여부",
#     "난자 채취 경과일",
#     "난자 해동 경과일",
#     "난자 혼합 경과일",
#     "배아 해동 경과일",
#     "불임 원인 - 정자 농도",
#     "불임 원인 - 정자 면역학적 요인",
#     "불임 원인 - 정자 운동성",
#     "불임 원인 - 정자 형태",
#     "불임 원인 - 여성 요인",
#     "불임 원인 - 자궁경부 문제",
#     "시술 시기 코드"
# ]

# X_train_encoded = X_train_encoded.drop(columns = drop_columns)
# X_test_encoded = X_test_encoded.drop(columns = drop_columns)

# # categorical_coulumns 에서 안쓰는 컬럼 삭제
# categorical_columns = [col for col in categorical_columns if col not in drop_columns]

# # numeric_coulumns 에서 안쓰는 컬럼 삭제
# numeric_columns = [col for col in numeric_columns if col not in drop_columns]

In [28]:
train.to_csv('./train_new.csv', index=False)
test.to_csv('./test_new.csv', index=False)

### Train

In [29]:
# CatBoost 모델 학습
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    cat_features=categorical_columns,
    random_seed=42,
    verbose=100
)

In [30]:
#X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
 #   X_train_encoded, y, test_size=0.2, random_state=42)

In [31]:
#CatBoost 모델 학습
model.fit(X_train_encoded, y)

0:	learn: 0.6403750	total: 868ms	remaining: 14m 26s
100:	learn: 0.4873635	total: 1m 21s	remaining: 12m 8s
200:	learn: 0.4851303	total: 2m 43s	remaining: 10m 50s
300:	learn: 0.4835223	total: 4m 7s	remaining: 9m 33s
400:	learn: 0.4823691	total: 5m 29s	remaining: 8m 12s
500:	learn: 0.4813897	total: 6m 53s	remaining: 6m 52s
600:	learn: 0.4804639	total: 8m 17s	remaining: 5m 30s
700:	learn: 0.4794956	total: 9m 41s	remaining: 4m 7s
800:	learn: 0.4786315	total: 11m 5s	remaining: 2m 45s
900:	learn: 0.4778475	total: 12m 29s	remaining: 1m 22s
999:	learn: 0.4770886	total: 13m 53s	remaining: 0us


### Predict

### Submission

In [32]:
# X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
#      X_train_encoded, y, test_size=0.2, random_state=42)

In [33]:
# # 검증 세트에서 예측
# y_val_pred = model.predict(X_val_split)
# y_val_proba = model.predict_proba(X_val_split)[:, 1]

# # 모델 평가
# print("\nClassification Report (Validation Set):")
# print(classification_report(y_val_split, y_val_pred))

# print("\nROC AUC Score (Validation Set):", roc_auc_score(y_val_split, y_val_proba))  # ROC AUC score

# print("\nConfusion Matrix (Validation Set):")
# print(confusion_matrix(y_val_split, y_val_pred))  # Confusion Matrix

# #항목 제거, 난자 나이 변수 생성, 결측치 0 생성 0.73796
# #항목 제거, 난자 나이 변수 생성, 결측치 0 생성, ICSI/IVF/DI 정의 0.7383548
# #항목 제거, 난자 나이 변수 생성, 결측치 0 생성, ICSI/IVF/DI 정의, 범주형 칼럼 생성 0.7373548468899807
# #xgboost 0.7356170370216362
# #ROC AUC Score (Validation Set): 0.7386579138877336
# #0.7511320718653633
# # 0.7518228298647986
# # 0.7519604645793005 + 배아 유형 제외
# #0.766970 -> 8 일때 catboost
# #0.7454377 -> 5 일때 앙상블
# #0.741711 -> xgboost
# #.7523970709535239 -> 로그 변환 후 표준화
# #0.75215 -> 결측값 기존
# #0.75220- > drop 안함.

In [34]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]
pred_proba

array([0.00163312, 0.0071384 , 0.15088373, ..., 0.47534767, 0.259004  ,
       0.00208589])

In [35]:
sample_submission = pd.read_csv('/content/drive/MyDrive/데이터 분석/LG Aimer/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [36]:
sample_submission.to_csv('/content/drive/MyDrive/데이터 분석/LG Aimer/baseline_submit.csv', index=False)